In [1]:
import binance_extract
import datetime as dt
import ta
import pandas as pd
import numpy as np
import datetime as dt
import requests
import os
import json


In [2]:
#To know if im in a docker enviroment or not
ON_DOCKER = os.environ.get('ON_DOCKER', False)

if ON_DOCKER:
    base_url = "http://fastapi:5000/"
else:
    base_url = "http://0.0.0.0:5000/"

ASSET_ID = "17679155-31be-407a-9808-b4f0b4d44955"   

BINANCE_URL = 'https://api.binance.com/api/v3/klines'


In [3]:


# Method to make API request to retrieve the last recorded price timestamp for a given asset ID
def api_request_last_time_stamp_from_asset_id(base_url, asset_id):
    try:
        # Make GET request to API endpoint
        response = requests.get(base_url + "assets/" + asset_id + "/last_price/")

        # Check if the request was successful
        if response.status_code == requests.codes.ok:
            print(response.status_code)
            # Convert JSON response to Python dictionary
            json_data = json.loads(response.content)
            # Normalize the dictionary and convert it to a Pandas DataFrame
            df = pd.json_normalize(json_data)
            # Return the Unix timestamp of the last recorded price for the specified asset ID
            return df['unix_time'].iloc[0]
        else:
            print(response.content)
        
    except:
        print("Failed api_request_last_time_stamp_from_asset_id")

# Method to make API request to retrieve prices for a given asset ID between two specified Unix timestamps
def api_request_get_prices_between_unix_time(base_url, asset_id, unix_time_start, unix_time_end):
    try:
        # Make GET request to API endpoint with query parameters for start and end Unix timestamps
        response = requests.get(base_url + "assets/" + asset_id + "/indicators_unix_between/", params={'unix_time_start': unix_time_start, 'unix_time_end': unix_time_end})

        # Check if the request was successful
        if response.status_code == requests.codes.ok:
            print(response.status_code)
            # Convert JSON response to Python dictionary
            json_data = json.loads(response.content)
            # Normalize the dictionary and convert it to a Pandas DataFrame
            df = pd.json_normalize(json_data)
            # Return the DataFrame containing the retrieved price data
            return df
        else:
            print(response.content)
        
    except:
        print("Failed api_request_get_prices_between_unix_time")

# Method to make a POST request to the specified API endpoint with a JSON payload
def post_method(url_path, data_dict):
    # Convert the dictionary to a JSON string using the `json` module
    json_payload = json.dumps(data_dict)
    try:
        # Send the POST request with the JSON payload
        response = requests.post(url_path, data=json_payload)

        # Check if the request was successful
        if response.status_code == requests.codes.created:
            # If successful, print the response content and convert it to a Pandas DataFrame
            print(response.content)
            json_data = json.loads(response.content)
            df = pd.json_normalize(json_data)
            return df
        else:
            # If unsuccessful, print the response content
            print(response.content)
    except:
        # If an exception occurs, print an error message
        print("Failed to make request")

# Method to create a new price for a given asset ID
def create_price_to_asset_id(base_url, asset_id, data_dict):
    # Call the `post_method` function with the appropriate URL and data dictionary
    return post_method(base_url + "assets/" + asset_id + "/prices/", data_dict)

# Method to create a new indicator for a given price ID
def create_indicator_to_price_id(base_url, price_id, data_dict):
    # Call the `post_method` function with the appropriate URL and data dictionary
    return post_method(base_url + "prices/" + price_id + "/indicators/", data_dict)


In [4]:
last_timestamp= api_request_last_time_stamp_from_asset_id(base_url,ASSET_ID)
last_timestamp

Failed api_request_last_time_stamp_from_asset_id


In [5]:
df_times= api_request_get_prices_between_unix_time(base_url,ASSET_ID,1673310,16733168499)
df_times

Failed api_request_get_prices_between_unix_time


In [6]:
# Define the JSON payload as a Python dictionary
data_dict = {
  "unix_time": "16733184",
  "open_price": "20000",
  "close_price": "22000",
  "low_price": "19500",
  "high_price": "22500",
  "volume": "72372787812",
  "qav": "10",
  "num_trades": "10",
  "taker_base_vol": "10",
  "taker_quote_vol": "10",
  "ignore": "10"
}

df2 = create_price_to_asset_id(base_url,ASSET_ID,data_dict)


Failed to make request


In [7]:
# Define the JSON payload as a Python dictionary
data_dict = {
  "name": "RSIe",
  "value": "12212",
  "unix_time": "1641013202"
}

df2 = create_indicator_to_price_id(base_url,"e4f0b3e7-9343-4986-a5e6-5ee429b3997b",data_dict)

Failed to make request


In [8]:
df = pd.read_csv("aux_folder/latest_minute.csv")


In [9]:
df = df.drop("close_time", axis=1)

In [10]:

df.head(3)

,unix_time,open_price,high_price,low_price,close_price,volume,qav,num_trades,taker_base_vol,taker_quote_vol,ignore
0,1641013202,46711.05,46740.56,46699.53,46740.55,12.71732,594022.681972,502,7.52260,351396.029446,0
1,1641013260,46740.56,46749.13,46725.72,46730.95,7.06492,330203.746976,295,3.89782,182185.587459,0
2,1641013320000,46730.95,46731.86,46715.00,46717.02,4.76270,222528.096882,328,2.28774,106889.770718,0


In [11]:
coun =0

# loop through all rows and convert each row to a JSON object
for index, row in df.iterrows():
    if coun ==1:
        break
    
    row_dict = row.to_dict()

    resp = create_price_to_asset_id(base_url,ASSET_ID,row_dict)

    print(resp)

 
    coun=coun+1
    print(row_dict)  # or do something else with the JSON object

Failed to make request
None
{'unix_time': 1641013202.0, 'open_price': 46711.05, 'high_price': 46740.56, 'low_price': 46699.53, 'close_price': 46740.55, 'volume': 12.71732, 'qav': 594022.6819719, 'num_trades': 502.0, 'taker_base_vol': 7.5226, 'taker_quote_vol': 351396.0294457, 'ignore': 0.0}


In [12]:
resp.head(2)

AttributeError: 'NoneType' object has no attribute 'head'

In [ ]:

def get_data_from_api(
    filename:str
    , symbol:str
    , initial_date:dt.datetime
    , limit_date:dt.datetime = dt.datetime.now()): 

    """ This function get the information from binance API
    """

    counter = 0 






In [ ]:
get_data_from_api(